In [55]:
import numpy as np
import cv2 
import scipy
import matplotlib.pylab as plt
import io

In [56]:
AruCo_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_1000)
AruCo_params = cv2.aruco.DetectorParameters_create()
board = cv2.aruco.GridBoard_create(1, 1, 0.08,0.005, AruCo_dict)


In [57]:
def draw_axis(img, corners, imgpts) :
    corner = tuple(corners[0].ravel().astype(int))
    img = cv2.line(img, corner, tuple(imgpts[0].ravel().astype(int)), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel().astype(int)), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel().astype(int)), (0,0,255), 5)
    return img

In [58]:
def write_text(img, pose, dy, text) :
    x0 = pose[0]
    y0 = pose[1]
    for i, line in enumerate(text.split('\n')) :
        y = y0 + i*dy
        cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.4, (0,255,0), 1) #### ปรับ รูปแบบตัวอักษร x,y,z

In [59]:
def augmented_image(frame,qr_img, pts_src, pts_dst):
    # Calculate Homography
    h,status  = cv2.findHomography(pts_src, pts_dst)
    # Warp source image to destination based on homography
    warped_image = cv2.warpPerspective(qr_img, h, (frame.shape[1],frame.shape[0]))      
    # Prepare a mask representing region to copy from the warped image into the original frame.
    mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(pts_dst), (255, 255, 255), cv2.LINE_AA)
    im_out = cv2.add(frame, warped_image, mask=cv2.bitwise_not(mask))
    im_out = cv2.add(im_out, warped_image)
    return im_out

In [60]:
####### ค่าจากการ calibation #######
params_dir = os.getcwd()+'/camera_params/monocular_camera_params/'
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

In [54]:
vdo = cv2.VideoCapture('./Dataset-1/left_output-1.avi')
tp_img = cv2.imread('./Templates/Template-1.png')
tp_gray = cv2.cvtColor(tp_img, cv2.COLOR_BGR2GRAY)
qr_img = cv2.imread('./QR-Code.jpg')
qr_img_size = qr_img.shape[:2]
src_points = np.float32([[0,0], [qr_img_size[1],0],[qr_img_size[1], qr_img_size[0]] ,[0, qr_img_size[0]] ])

while vdo.isOpened():
    ret,frame = vdo.read()
    ############## ปรับให้ video สว่างขึ้น ###############
    kernal = np.array([ [0,0,0], [0,1.7,0], [0,0,0]] )
    bright = cv2.filter2D(frame,-1, kernal) 
    vdo_gray = cv2.cvtColor(bright, cv2.COLOR_BGR2GRAY)
    
    if ret :
        
        sift = cv2.SIFT_create()
        Bf = cv2.BFMatcher()

        template_kpts, template_desc = sift.detectAndCompute(tp_gray, None)
        query_kpts, query_desc = sift.detectAndCompute(vdo_gray, None)
        matches = Bf.knnMatch(template_desc, query_desc, k=2)
        good_matches = list()
        good_matches_list = list()

        for m, n in matches :
            if m.distance < 0.7*n.distance :
                good_matches.append(m)
                good_matches_list.append([m])
    
        if len(good_matches) > 16 :
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 1.3) # H RANSAC
          
            h, w = tp_img.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)
            
            detection_img = cv2.polylines(frame, [np.int32(transformed_box)], True, (2,20,200), 2, cv2.LINE_AA)
            draw_img = cv2.drawMatchesKnn(tp_img, template_kpts, detection_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
            ################# warp qr code ##################################
            augmented = augmented_image(detection_img, qr_img, src_points, transformed_box)
            markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(augmented, AruCo_dict, parameters = AruCo_params)
            ####### ตรวจจับ qr code เพื่อทำการระบุพิกัดแกน x,y,z #######
            if len(markerCorners) > 0: ### จำนวน qr ที่ตรวจจับ 
                xyz = cv2.aruco.drawDetectedMarkers(augmented, markerCorners)
                rvecs, tvecs, points = cv2.aruco.estimatePoseSingleMarkers(markerCorners , 0.08, K, dist)
                for (rvec, tvec, id, corner) in zip(rvecs, tvecs, markerIds, markerCorners) :
                    xyz = cv2.aruco.drawAxis(augmented, K, dist, rvec, tvec, 0.08)
                    x = tvec[0,0]
                    y = tvec[0,1]
                    z = tvec[0,2]
                    text = "x = {:.2f} y = {:.2f} z = {:.2f}".format( x, y, z)
                    cX = (corner[0,0][0] + corner[0,2][0]) / 2
                    cY = (corner[0,0][1] + corner[0,2][1]) / 2
                    write_text(frame, (cX, cY), 20, text)
                ret, brvec, btvec = cv2.aruco.estimatePoseBoard(markerCorners, markerIds, board, K, dist, rvecs, tvecs) 
            if ret :
                   xyz = cv2.aruco.drawAxis(augmented, K, dist, brvec, btvec, 0.08)

            cv2.imshow('VIDEO',frame)

        if cv2.waitKey(int(1000/30)) & 0xFF == ord('q') : 
            break
    else :
        break

vdo.release()
cv2.destroyAllWindows()